In [1]:
import neptune.new as neptune
from utils import *
import numpy as np
import plotly.express as px

2022-11-16 12:29:41.864625: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 12:30:24.504139: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-16 12:30:26.606794: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/insane/.cache/pypoetry/virtualenvs/3d-reconstruction-GT1uMocM-py3.10/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-16 12:30:26.606842: W tensorflow/stream_executor/platform/default/dso_loader.cc:64]

In [3]:
project = neptune.get_project(
    name='rahatzamancse/MPSE-on-LMNET-dataset', 
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ=='
)

# Get dashboard with runs contributed by 'sophia' tagged 'cycleLR'
run_table_df = project.fetch_runs_table(
    owner='rahatzamancse',
).to_pandas()

run_table_df.columns

https://app.neptune.ai/rahatzamancse/MPSE-on-LMNET-dataset/
Remember to stop your project once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/project#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


NeptuneConnectionLostException: 
[95m
----NeptuneConnectionLostException---------------------------------------------------------
[0m
The connection to the Neptune server was lost.
If you are using the asynchronous (default) connection mode, Neptune continues to locally track your metadata and continuously tries to re-establish a connection to the Neptune servers.
If the connection is not re-established, you can upload your data later with the Neptune Command Line Interface tool:
    [95mneptune sync -p workspace_name/project_name[0m

What should I do?
    - Check if your computer is connected to the internet.
    - If your connection is unstable, consider working in offline mode:
        [96mrun = neptune.init(mode="offline")[0m

You can find detailed instructions on the following doc pages:
    - https://docs.neptune.ai/you-should-know/connection-modes#offline
    - https://docs.neptune.ai/you-should-know/connection-modes#uploading-offline-data

You may also want to check the f

In [ ]:
run_table_df['parameters/PROJECTION/PROJ_TYPE']

In [ ]:
run_table_df[
    (run_table_df['parameters/DATASET'] == 'ShapeNet:sofa:d8c748ced5e5f2cc7e3820d17093b7c2') 
    & (run_table_df['parameters/INITIAL_EMBEDDING'] == False)
    & (run_table_df['parameters/MPSE/BATCH_SIZE'] == 2048)
    & (run_table_df['parameters/MPSE/SMART_INITIALIZATION'] == True)
    & (run_table_df['parameters/MPSE/VARIABLE_PROJECTION'] == False)
    & (run_table_df['parameters/N_POINTS'] == 512)
    & (run_table_df['parameters/N_PERSPECTIVE'] == 4)
    & (run_table_df['parameters/PROJECTION/POINT_IN_ATLEAST'] == 3)
    & (run_table_df['parameters/PROJECTION/PROJ_TYPE'] == 'atleast_in_n_persp')
]

In [ ]:
len(run_table_df[
    (run_table_df['parameters/DATASET'] == 'ShapeNet:sofa:d8c748ced5e5f2cc7e3820d17093b7c2') 
    & (run_table_df['parameters/INITIAL_EMBEDDING'] == False)
    & (run_table_df['parameters/MPSE/BATCH_SIZE'] == 2048)
    & (run_table_df['parameters/MPSE/SMART_INITIALIZATION'] == True)
    & (run_table_df['parameters/MPSE/VARIABLE_PROJECTION'] == False)
    & (run_table_df['parameters/N_POINTS'] == 512)
    & (run_table_df['parameters/N_PERSPECTIVE'] == 4)
    & (run_table_df['parameters/PROJECTION/POINT_IN_ATLEAST'] == 3)
    & (run_table_df['parameters/PROJECTION/PROJ_TYPE'] == 'atleast_in_n_persp')
])

In [ ]:
info_table = run_table_df

In [69]:
run_id = "MPSELMNET-509"

run = neptune.init(
    project='rahatzamancse/MPSE-on-LMNET-dataset',
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ==',
    run=run_id, mode="read-only"
)

# print("Ground Truth")
# run["GT/X"].download('/tmp/')
# points = np.load('/tmp/X.npy')
# print(points.min(), points.max())
# fig = plot_3D([points])
# fig.write_html("GT.html")
# fig.show()

print("Embedding")
run["Results/Embedding_X"].download('/tmp/')
embedding = np.load('/tmp/Embedding_X.npy')
# fig = plot_3D([embedding], colors=['red'])
embedding = np.dot(embedding, getRotationMatrix(90, 'x'))
embedding = np.dot(embedding, getRotationMatrix(0, 'y'))
embedding = np.dot(embedding, getRotationMatrix(0, 'z'))
# embedding = np.dot(embedding, getRotationMatrix(180, 'y'))
fig = plot_3D_paper([embedding], point_size=2, colors=['#0c3a6d'])
# fig.write_html("Embedding.html")
fig.show()



# all_trans_mats = []
# for metric in ['4PointSample', 'Global-RMSE-opt']:
#     path = "Results/Alignment/" + metric + '/transformation_mat'
#     run[path].download(f'/tmp/')
#     trans_mats = np.load('/tmp/transformation_mat.npy')
#     all_trans_mats.append(trans_mats)

# # 4Point
# print("4PointSample :", run['Results/Alignment/4PointSample/error'].fetch())
# aligned_embedding = apply_transformation(embedding, all_trans_mats[0])
# fig = plot_3D([points, aligned_embedding])
# fig.write_html("4pointaligned.html")
# fig.show()

# # Global
# print("Global-RMSE-opt :", run['Results/Alignment/Global-RMSE-opt/error'].fetch())
# aligned_embedding = apply_transformation(embedding, all_trans_mats[1])
# fig = plot_3D([points, aligned_embedding])
# fig.write_html("rmsealigned.html")
# fig.show()

# print("Projections")
# run["GT/Projections"].download('/tmp/')
# projections = np.load('/tmp/Projections.npy')
# print(projections.min(), projections.max())

run.stop()

parameter `run` is deprecated, use `with_id` instead. We'll end support of it in `neptune-client==1.0.0`.
https://app.neptune.ai/rahatzamancse/MPSE-on-LMNET-dataset/e/MPSELMNET-509
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Embedding


Shutting down background jobs, please wait a moment...
Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/rahatzamancse/MPSE-on-LMNET-dataset/e/MPSELMNET-509


In [ ]:
# # N_Points 256
# chair
# variable proj
# raytracing

# data = {
#     'proj_3': 'MPSEMDS-6154',
#     'proj_4': 'MPSEMDS-6156',
#     'proj_5': 'MPSEMDS-6158',
# }


def ghurao(myPoints):
    
    rots_toilet = [
        ('x', 100),
        ('z', -140),
        ('x', 35),
        ('z', 30),
        ('x', 10),
    ]
    
    rots_desk = [
        ('x', 100),
        ('z', -140),
        ('x', 35),
        ('z', 30),
        # ('x', 10),
    ]
    
    rots_chair = [
        ('x', 100),
        ('z', -140),
        ('x', 35),
        ('z', -30),
        ('x', 10),
    ]
    
    rots = rots_chair
    
    myPoints -= myPoints.mean(axis=0)

    
    for ax, deg in rots:
        myPoints = apply_transformation(myPoints, getRotationMatrix(deg, ax))
        myPoints -= myPoints.mean(axis=0)
    
    return myPoints


chair_data = {
    'proj_3': 'MPSEMDS-6107',
    'proj_4': 'MPSEMDS-6156',
    'proj_5': 'MPSEMDS-6111',
}

desk_data = {
    'proj_3': 'MPSEMDS-6129',
    'proj_4': 'MPSEMDS-6131',
    'proj_5': 'MPSEMDS-6133',
}

toilet_data = {
    'proj_3': 'MPSEMDS-7031',
    'proj_4': 'MPSEMDS-6486',
    'proj_5': 'MPSEMDS-7049',
}



data = desk_data

print("Ground Truth")
run = neptune.init(
    project='rahatzamancse/MPSE-mds',
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ==',
    run=list(data.values())[0], mode="read-only"
)

run["GT/X"].download('/tmp/')
points = np.load('/tmp/X.npy')
points = ghurao(points)


fig = plot_3D_paper([points])
fig.write_html("toilet_data.html")
fig.show()

run.stop()

for i,val in data.items():
    print(i)
    run = neptune.init(
        project='rahatzamancse/MPSE-mds',
        api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ==',
        run=val, mode="read-only"
    )
    
    run["GT/X"].download('/tmp/')
    points = np.load('/tmp/X.npy')
    points = ghurao(points)

    run["Results/Embedding_X"].download('/tmp/')
    embedding = np.load('/tmp/Embedding_X.npy')
    trans_mats, align_error = get_optimal_trans_mat(embedding, points, iterations=1600)
    aligned_embedding = apply_transformation(embedding, trans_mats)
    fig = plot_3D_paper([aligned_embedding], colors=['red'])
    fig.show()
    
    run.stop()

In [ ]:
run_id = "MPSEMDS-6107"

run = neptune.init(
    project='rahatzamancse/MPSE-mds',
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ==',
    run=run_id, mode="read-only"
)

print("Ground Truth")
run["GT/X"].download('/tmp/')
points = np.load('/tmp/X.npy')
fig = plot_3D_paper([points])
fig.show()

# print("Embedding")
# run["Results/Embedding_X"].download('/tmp/')
# embedding = np.load('/tmp/Embedding_X.npy')
# fig = plot_3D_paper([embedding], colors=['red'])
# fig.show()

all_trans_mats = []
for metric in ['4PointSample', 'Global-RMSE-opt']:
    path = "Results/Alignment/" + metric + '/transformation_mat'
    run[path].download(f'/tmp/')
    trans_mats = np.load('/tmp/transformation_mat.npy')
    all_trans_mats.append(trans_mats)

# Global
print("Global-RMSE-opt :", run['Results/Alignment/Global-RMSE-opt/error'].fetch())
aligned_embedding = apply_transformation(embedding, all_trans_mats[1])
fig = plot_3D_paper([aligned_embedding], colors=['red'])
fig.show()

run.stop()

In [ ]:
run_id = "MPSEMDS-6156"

run = neptune.init(
    project='rahatzamancse/MPSE-mds',
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3NDk5MWVkNS0wMDg4LTRmNTktYWQyZC0zNzUyNTg0MTY1MGYifQ==',
    run=run_id, mode="read-only"
)

print("Ground Truth")
run["GT/X"].download('/tmp/')
points = np.load('/tmp/X.npy')
fig = plot_3D_paper([points])
fig.show()

# print("Embedding")
# run["Results/Embedding_X"].download('/tmp/')
# embedding = np.load('/tmp/Embedding_X.npy')
# fig = plot_3D_paper([embedding], colors=['red'])
# fig.show()

all_trans_mats = []
for metric in ['4PointSample', 'Global-RMSE-opt']:
    path = "Results/Alignment/" + metric + '/transformation_mat'
    run[path].download(f'/tmp/')
    trans_mats = np.load('/tmp/transformation_mat.npy')
    all_trans_mats.append(trans_mats)

# Global
print("Global-RMSE-opt :", run['Results/Alignment/Global-RMSE-opt/error'].fetch())
aligned_embedding = apply_transformation(embedding, all_trans_mats[1])
fig = plot_3D_paper([aligned_embedding], colors=['red'])
fig.show()

run.stop()

In [ ]:
min_dist, best_t = get_4pointsample_transform_mat(embedding, points, dist_agg_fn=None, t_sample=10000000)

# dresser', 'monitor', 'night_stand', 'sofa', 'table', 'toilet
aligned_points = apply_transformation(embedding, best_t)

plot_3D([points, aligned_points])

# For each projections applied to ground truth

In [ ]:
p = []
for i in range(len(projections)):
    tranformed_points = apply_transformation(points, projections[i])
    tranformed_points -= tranformed_points.mean(axis=0)
    tranformed_points[:, 2] = 0
    p.append(tranformed_points)
    fig = plot_3D([tranformed_points])
    fig.write_html(f"perspecive_{i}.html")
    fig.show()
    # fig.write_image("fig1.png")
    fig.show()